In [1]:
from __future__ import print_function
import os
import csv
import collections
import string
from time import time
import subprocess
import io
import re
import urllib2
from bs4 import BeautifulSoup
from multiprocessing import Process, Lock, Pool

In [2]:
umlsMetDir = "/scratch/saeid/umls/2015AA/2015AA/META/"
mrrel = os.path.join(umlsMetDir, "MRREL.RRF")
mrsty = os.path.join(umlsMetDir, "MRSTY.RRF")
mrconso = os.path.join(umlsMetDir, "MRCONSO.RRF")
indexDir = "/scratch/index/indri_5_7/pmc_mod/"
projDir = "/home/fj9124/projects/ir/seq_kb_ir/"
configsDir = os.path.join(projDir, "configs")
cds14configsDir = os.path.join(configsDir, "cds14")
cds14Indriconfigs = os.path.join(cds14configsDir, "indriRunQuery.cfg")
print("mrrel =", mrrel)
print("mrsty =", mrsty)

mrrel = /scratch/saeid/umls/2015AA/2015AA/META/MRREL.RRF
mrsty = /scratch/saeid/umls/2015AA/2015AA/META/MRSTY.RRF


In [7]:
def readRrfFile(rrfFileName, indices):
    print("RRF file Name =", rrfFileName)
    i0 = indices[0]
    i1 = indices[1]
    print("indices =", i0, i1)
    t0 = time()
    print("reading RRF file and store in a dict of lists")
    rrf = dict()
    with open (rrfFileName, 'r') as rrfFile:
        reader = csv.reader(rrfFile, delimiter='|')
        for row in reader:
            #print(row)
            #print(rrfFileName, mrconso)
            #print(rrfFileName == mrconso)
            if rrfFileName == mrconso:
                if row[1] == "ENG" and row[2] == "P" and row[4] == "PF" and row[6] == "Y":
                    #print(row[1], row[2], row[4], row[6])
                    #break
                    row[i1] = re.sub("[\(\[].*?[\)\]]", "", row[i1])
                    #print(row[i1])
                    if row[i1].isalpha():
                        row[i1] = row[i1].lower().strip()
                        #print(row[i1])
                        #break
                    else:
                        continue
                else:
                    continue
                    
            #print (row)
            if row[i0] in rrf:
                rrf[row[i0]].add(row[i1])
            else:
                rrf[row[i0]] = set([row[i1]])
    print("size of readed matrix =", len(rrf))
    print("done in %0.3f minutes." % ((time() - t0)/60))
    return(rrf)

In [10]:
relConcepts = readRrfFile(mrrel, [0, 4])

RRF file Name = /scratch/saeid/umls/2015AA/2015AA/META/MRREL.RRF
indices = 0 4
reading RRF file and store in a dict of lists
size of readed matrix = 2255894
done in 1.390 minutes.


In [11]:
t0 = time()
print("reading MRREL.RRF file and store in a dict of list with counter on the number of relationship links")
for c1, c2 in relConcepts.iteritems():
    relConcepts[c1] = collections.Counter( c2 )
print("number of concepts with relationship links to the others =", len(relConcepts))
print("done in %0.3f minutes." % ((time() - t0)/60))

reading MRREL.RRF file and store in a dict of list with counter on the number of relationship links
number of concepts with relationship links to the others = 2255894
done in 0.514 minutes.


In [12]:
styConcepts = readRrfFile(mrsty, [0, 1])

RRF file Name = /scratch/saeid/umls/2015AA/2015AA/META/MRSTY.RRF
indices = 0 1
reading RRF file and store in a dict of lists
size of readed matrix = 2296350
done in 0.441 minutes.


In [8]:
conceptNames = readRrfFile(mrconso, [0, 14])

RRF file Name = /scratch/saeid/umls/2015AA/2015AA/META/MRCONSO.RRF
indices = 0 14
reading RRF file and store in a dict of lists
size of readed matrix = 151742
done in 0.483 minutes.


In [14]:
for i, (r, c) in enumerate(relConcepts.iteritems()):
    print ("relevant concepts =", r, c)
    if i > 10:
        break
print("-----------------------------------------------")
for i, (s, c1) in enumerate(styConcepts.iteritems()):
    print ("semantic types of concepts =", s, c1)
    if i > 10:
        break
print("-----------------------------------------------")
for i, (s, c1) in enumerate(conceptNames.iteritems()):
    print ("concepts and their names =", s, c1)
    if i > 10:
        break

relevant concepts = C3098694 Counter({'C2270445': 1})
relevant concepts = C1912735 Counter({'C0752036': 1})
relevant concepts = C3812400 Counter({'C3669017': 1})
relevant concepts = C3608447 Counter({'C3608446': 1})
relevant concepts = C1881803 Counter({'C0002763': 1, 'C1880653': 1})
relevant concepts = C1881802 Counter({'C0003211': 1, 'C1880653': 1})
relevant concepts = C1881801 Counter({'C1518174': 1, 'C1334740': 1, 'C0014597': 1, 'C1513959': 1, 'C0334227': 1, 'C1707292': 1, 'C1514098': 1, 'C1510779': 1, 'C1513973': 1, 'C1513183': 1, 'C0014609': 1, 'C1881876': 1, 'C0206696': 1, 'C0334037': 1, 'C0022790': 1, 'C0597032': 1, 'C1707933': 1, 'C1518245': 1, 'C1518246': 1, 'C1514101': 1, 'C1522040': 1, 'C0334277': 1})
relevant concepts = C1881800 Counter({'C1332746': 1, 'C1708995': 1, 'C0815043': 1, 'C1333346': 1, 'C0007082': 1, 'C1335977': 1, 'C1513177': 1})
relevant concepts = C1881807 Counter({'C0003232': 1, 'C1880653': 1})
relevant concepts = C1881806 Counter({'C1516000': 1, 'C1880653':

In [15]:
response = urllib2.urlopen('https://metamap.nlm.nih.gov/Docs/SemanticTypes_2013AA.txt')
html = response.read()
f = io.StringIO(unicode(html))
reader = csv.reader(f, delimiter='|', skipinitialspace=True,)
sty_l = list(reader)
#print(sty_l)
sty_d_abbr = {i[0]:i[1] for i in sty_l}
sty_d_str = {i[2]:i[1] for i in sty_l}
print(sty_d_abbr)
print(sty_d_str)

{'popg': 'T098', 'inpo': 'T037', 'elii': 'T196', 'idcn': 'T078', 'neop': 'T191', 'drdd': 'T203', 'inpr': 'T170', 'resd': 'T075', 'evnt': 'T051', 'phsu': 'T121', 'blor': 'T029', 'hops': 'T131', 'menp': 'T041', 'phsf': 'T039', 'humn': 'T016', 'ftcn': 'T169', 'anim': 'T008', 'food': 'T168', 'grpa': 'T102', 'rept': 'T014', 'prog': 'T097', 'hcpp': 'T068', 'vtbt': 'T010', 'lbtr': 'T034', 'edac': 'T065', 'ocdi': 'T090', 'crbs': 'T088', 'chvf': 'T120', 'anab': 'T190', 'tisu': 'T024', 'inbe': 'T055', 'tmco': 'T079', 'clas': 'T185', 'acab': 'T020', 'dsyn': 'T047', 'fish': 'T013', 'horm': 'T125', 'bact': 'T007', 'eehu': 'T069', 'grup': 'T096', 'bacs': 'T123', 'enty': 'T071', 'chem': 'T103', 'vita': 'T127', 'resa': 'T062', 'medd': 'T074', 'chvs': 'T104', 'mcha': 'T066', 'pros': 'T094', 'cell': 'T025', 'fndg': 'T033', 'sbst': 'T167', 'geoa': 'T083', 'celf': 'T043', 'bird': 'T012', 'diap': 'T060', 'celc': 'T026', 'hcro': 'T093', 'clnd': 'T200', 'orch': 'T109', 'clna': 'T201', 'bodm': 'T122', 'patf':

In [16]:
response = urllib2.urlopen('http://trec-cds.appspot.com/topics2014.xml')
html = response.read()
#print(html)
soup = BeautifulSoup(html, "lxml")
origQueries = {fat.attrs['number']: fat.find('summary').next for fat in soup.findAll('topic')}
print((origQueries))

{'24': u' 33-year-old male presents with severe abdominal pain one week after a bike accident in which he sustained abdominal trauma. He is hypotensive and tachycardic, and imaging reveals a ruptured spleen and intraperitoneal hemorrhage.', '25': u'8-year-old boy with decreased level of consciousness 30 minutes after falling from his bike and hitting his head. He was not wearing a helmet. In the ER he was bradycardic and hypertensive, and the neurosurgeon strongly advised immediate treatment.', '26': u"Group traveling to the Amazon rainforest, including 3 pregnant women. All members' immunizations are up-to-date but they require malaria prophylaxis.", '27': u'21-year-old male with dozens of small polyps found on colonoscopy. Family history significant for both older siblings having total colectomies after colonoscopy revealed hundreds of adenomas.', '20': u'32-year-old unbelted driver status post car accident. Has multiple extremity fractures and severe abdominal pain with rebound and 

In [17]:
def text2umlsConcepts(origQueries_l):
    #return(type(origQuery))
    origQuery = origQueries_l[1]
    out = subprocess.check_output(["/home/fj9124/thirdPartyProgs/public_mm/bin/testapi.sh", "-AIy+", "-V USAbase", str(origQuery)])
    #print((out))
    f = io.StringIO(unicode(out))
    reader = csv.reader(f, delimiter=':', skipinitialspace=True,)
    #reader1 = reader
    extractedConcepts = [row[1] for row in reader if row[0] in set(['Concept Name', 'Concept Id', 'Semantic Types'])]
    #print(len(extractedConcepts))
    extractedConcepts1 = set([(extractedConcepts[3*i], extractedConcepts[3*i+1].strip().lower(), extractedConcepts[3*i+2].strip(']').strip('[')) for i in range(0, len(extractedConcepts)/3)])
    return([origQueries_l[0], extractedConcepts1])
    

In [18]:
print("getting original concepts")
origQueries_l = [[k, v] for k, v in origQueries.iteritems()]

pool = Pool(processes=10) 
qidConcepts = (pool.map(text2umlsConcepts, origQueries_l))
print(qidConcepts)

getting original concepts
[['24', set([('C0443318', 'sustained', 'ftcn'), ('C0439508', '/year', 'tmco'), ('C0205082', 'severe', 'qlco'), ('C0278140', 'severe pain', 'sosy'), ('C0038000', 'ruptured spleen', 'inpo'), ('C0332174', '/week', 'tmco'), ('C0848377', 'abdominal trauma', 'inpo'), ('C0011923', 'imaging', 'diap'), ('C0857353', 'hypotensive', 'patf'), ('C0687676', 'after', 'tmco'), ('C0701802', 'bike accident', 'inpo'), ('C0150312', 'present', 'qnco'), ('C0000737', 'abdominal pain', 'sosy'), ('C0039231', 'tachycardia', 'fndg'), ('C0443289', 'reveal', 'qlco'), ('C0019065', 'intraperitoneal hemorrhage', 'patf'), ('C0000726', 'abdominal', 'blor'), ('C0205447', 'one', 'qnco'), ('C0580836', 'old', 'tmco'), ('C1706180', 'male', 'qlco')])], ['25', set([('C0019602', 'his', 'aapp, bacs, phsu'), ('C1821572', 'decreased level of consciousness', 'fndg'), ('C0018884', 'helmet', 'mnob'), ('C0087111', 'treatment', 'topp'), ('C0018670', 'head', 'blor'), ('C0205253', 'immediate', 'tmco'), ('C043950

In [19]:
print("convert qidConcepts to a dictionary formatted container")
qidConcepts_d = {qidConcs[0]: list(qidConcs[1]) for qidConcs in qidConcepts} 
print(qidConcepts_d)

convert qidConcepts to a dictionary formatted container
{'24': [('C0443318', 'sustained', 'ftcn'), ('C0439508', '/year', 'tmco'), ('C0205082', 'severe', 'qlco'), ('C0278140', 'severe pain', 'sosy'), ('C0038000', 'ruptured spleen', 'inpo'), ('C0332174', '/week', 'tmco'), ('C0848377', 'abdominal trauma', 'inpo'), ('C0011923', 'imaging', 'diap'), ('C0857353', 'hypotensive', 'patf'), ('C0687676', 'after', 'tmco'), ('C0701802', 'bike accident', 'inpo'), ('C0150312', 'present', 'qnco'), ('C0000737', 'abdominal pain', 'sosy'), ('C0039231', 'tachycardia', 'fndg'), ('C0443289', 'reveal', 'qlco'), ('C0019065', 'intraperitoneal hemorrhage', 'patf'), ('C0000726', 'abdominal', 'blor'), ('C0205447', 'one', 'qnco'), ('C0580836', 'old', 'tmco'), ('C1706180', 'male', 'qlco')], '25': [('C0019602', 'his', 'aapp, bacs, phsu'), ('C1821572', 'decreased level of consciousness', 'fndg'), ('C0018884', 'helmet', 'mnob'), ('C0087111', 'treatment', 'topp'), ('C0018670', 'head', 'blor'), ('C0205253', 'immediate', 

In [20]:
goodSty = ["Clinical Drug", "Disease or Syndrome", "Injury or Poisoning", "Pharmacologic Substance", "Sign or Symptom", "Therapeutic or Preventive Procedure"]
goodTui = [sty_d_str[gSty] for gSty in goodSty]
print (goodTui)

['T200', 'T047', 'T037', 'T121', 'T184', 'T061']


In [66]:
print("filter good original concept")
qOrigConcepts_good_d = dict()
for q, Concepts_d in qidConcepts_d.iteritems():
    for concProps in Concepts_d:
        for sty_ in concProps[2].split(','):
            if sty_d_abbr[sty_.strip()] in goodTui:
                #print(sty_, sty_d_abbr[sty_.strip()])
                if q in qOrigConcepts_good_d:
                    qOrigConcepts_good_d[q].append(concProps)
                else:
                    qOrigConcepts_good_d[q]= [concProps]
print(qOrigConcepts_good_d)

filter good original concept
{'24': [('C0278140', 'severe pain', 'sosy'), ('C0038000', 'ruptured spleen', 'inpo'), ('C0848377', 'abdominal trauma', 'inpo'), ('C0701802', 'bike accident', 'inpo'), ('C0000737', 'abdominal pain', 'sosy')], '25': [('C0019602', 'his', 'aapp, bacs, phsu'), ('C0087111', 'treatment', 'topp')], '26': [('C0020971', 'immunizations', 'topp'), ('C0199244', 'malaria prophylaxis', 'topp')], '27': [('C0009274', 'colectomies', 'topp')], '20': [('C1579887', 'car accident', 'inpo'), ('C0000924', 'accident, nos', 'inpo'), ('C0016655', 'multiple fractures', 'inpo'), ('C0234246', 'rebound abdominal pain', 'sosy')], '21': [('C0003862', 'arthralgias', 'sosy'), ('C0015672', 'fatigue', 'sosy')], '22': [('C0694551', 'abdominal pain, right lower quadrant', 'sosy'), ('C0232462', 'decreased appetite', 'sosy')], '23': [('C0239134', 'productive cough', 'sosy'), ('C0030054', 'oxygen', 'elii, phsu'), ('C0231835', 'tachypnea', 'sosy')], '28': [('C0002453', 'amenorrhea', 'dsyn'), ('C0567

In [108]:
print("extract 1st layer good related concept")
countLim = 1
qRelConcepts_good_1 = dict()
qRelConcepts_good_mutualCount_1 = dict()
for qNo, origConcs in qOrigConcepts_good_d.iteritems():
    for origConc in origConcs:
        #print(qidConcepts_d)
        if origConc[0] in relConcepts:
            relConcepts1conc = relConcepts[origConc[0]]
            #print(origConc, relConcepts1conc)
            for relCui in relConcepts1conc.keys():
                # if the concept has an acceptable name and good semantic type
                if relCui in conceptNames and bool(styConcepts[relCui]&set(goodTui)):
                    t = (relCui, list(conceptNames[relCui])[0], ','.join(styConcepts[relCui]))
                    #print(qNo)
                    #break
                    if qNo in qRelConcepts_good_1:
                        #print("----")
                        qRelConcepts_good_1[qNo].append(t)
                    else:
                        qRelConcepts_good_1[qNo] = [t]
    if qNo in qRelConcepts_good_1:
        l = qRelConcepts_good_1[qNo]
        qRelConcepts_good_mutualCount_1[qNo] = list(set([x for x in l if l.count(x) > countLim]))
        #print(qNo, list(set([(l.count(x), list(x)[1]) for x in l if l.count(x) > 1])))
#qWeightsLayerConcepts[qNo].append ([0.05, origConcs])
print(qRelConcepts_good_mutualCount_1)

extract 1st layer good related concept
{'24': [('C0009938', 'contusions', 'T037'), ('C3203359', 'rupture', 'T037'), ('C0016736', 'frostbite', 'T037')], '25': [], '26': [], '27': [], '20': [], '21': [('C0030193', 'pain', 'T184'), ('C1457887', 'symptoms', 'T184'), ('C0018681', 'headache', 'T184')], '22': [], '23': [('C0242706', 'hyperoxia', 'T184'), ('C0010200', 'coughing', 'T184')], '28': [], '29': [('C0057558', 'desogestrel', 'T109,T121,T125'), ('C0025506', 'mestranol', 'T109,T121'), ('C0035579', 'rickets', 'T047'), ('C0029456', 'osteoporosis', 'T047'), ('C0013336', 'dwarfism', 'T019,T047'), ('C0017601', 'glaucoma', 'T047'), ('C0037199', 'sinusitis', 'T047'), ('C0040822', 'tremor', 'T184')], '1': [], '5': [], '4': [], '7': [('C0024977', 'mazindol', 'T109,T121'), ('C0011175', 'dehydration', 'T047')], '6': [('C0023520', 'leukodystrophy', 'T047'), ('C0040822', 'tremor', 'T184'), ('C0031485', 'phenylketonurias', 'T047'), ('C1457887', 'symptoms', 'T184'), ('C0011175', 'dehydration', 'T047')

In [109]:
print("combine different level of concepts")
qWeightsLayerConcepts = dict()
for qNo, qText in origQueries.iteritems():
    qWeightsLayerConcepts[qNo] = [[0.95, qText]]
    if qNo in qidConcepts_good_d:
        origConcs_good = qidConcepts_good_d[qNo]
        qWeightsLayerConcepts[qNo].append ([0.05, origConcs_good])
    if qNo in qRelConcepts_good_1:
        relConcepts_good_1 = qRelConcepts_good_1[qNo]
        qWeightsLayerConcepts[qNo].append ([0.01, relConcepts_good_1])

print(qWeightsLayerConcepts)

combine different level of concepts
{'24': [[0.95, u' 33-year-old male presents with severe abdominal pain one week after a bike accident in which he sustained abdominal trauma. He is hypotensive and tachycardic, and imaging reveals a ruptured spleen and intraperitoneal hemorrhage.'], [0.05, [('C0278140', 'severe pain', 'sosy'), ('C0038000', 'ruptured spleen', 'inpo'), ('C0848377', 'abdominal trauma', 'inpo'), ('C0701802', 'bike accident', 'inpo'), ('C0000737', 'abdominal pain', 'sosy')]], [0.01, [('C0020532', 'hypersplenism', 'T047'), ('C0206369', 'splenosis', 'T047'), ('C3203359', 'rupture', 'T037'), ('C0012691', 'dislocations', 'T037'), ('C0043246', 'laceration', 'T037'), ('C0035361', 'retropneumoperitoneum', 'T037'), ('C0004760', 'dysbarism', 'T037'), ('C0009938', 'contusions', 'T037'), ('C3203359', 'rupture', 'T037'), ('C0013142', 'drowning', 'T037'), ('C0016736', 'frostbite', 'T037'), ('C0016658', 'fracture', 'T037'), ('C0000924', 'accidents', 'T037'), ('C0013456', 'earache', 'T1

In [110]:
def generateIndriQueries(queryfile, qWeightsLayersConcepts, indexDir, goodTui):
    print("queryfile =", queryfile)
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    with open(queryfile, "w") as f:
        f.write("<parameters>\n<index>" + indexDir + "</index>\n")
        for q, wLaysConcs in qWeightsLayersConcepts.iteritems():
            f.write("<query>\n")
            f.write("<number>" + str(q) + "</number>\n")
            f.write("<text>\n")
            f.write("\t#weight(\n")
            for i, wLayConcs in enumerate(wLaysConcs):
                #print("---> ", wLayConcs)
                wLay = wLayConcs[0]
                layConcs = wLayConcs[1]
                if i == 0:
                    layConcs = regex.sub(' ', layConcs)
                    #layConcs = str(layConcs).translate(table, string.punctuation)
                    f.write("\t\t" + str(wLay) + " #combine(" + layConcs + ")\n" )
                else:
                    concs = []
                    for concProps in layConcs:
                        concs.append(regex.sub(' ', concProps[1]))
                    if len(concs) > 0:
                        f.write("\t\t" + str(wLay) + " #weight(\n" )
                        for conc in concs:
                            #print(conc)
                            #conc = concProps[1].translate(None, string.punctuation)
                            f.write("\t\t\t" + str(1.0) + " #weight(\n" )
                            f.write("\t\t\t\t" + str(0.8) + " #combine(" + conc + ")\n")
                            f.write("\t\t\t\t" + str(0.1) + " #combine(" )
                            concStrs = conc.split()
                            concStrs1 = [concStrs[ii] + " " + concStrs[ii+1] for ii in range(0, len(concStrs)-1)]
                            if len(concStrs1) > 0:
                                for concStr in concStrs1:
                                    f.write("#od4(" + concStr.strip() + ")")
                            else:
                                f.write(conc)
                            f.write(")\n")
                            f.write("\t\t\t\t" + str(0.1) + " #uw17(" + conc + ")\n")
                            f.write("\t\t\t)\n" )
                        f.write("\t\t)\n" )
            f.write("\t)\n")
            f.write("</text>\n")
            f.write("</query>\n")
        f.write("<rule>method:two</rule>\n")
        f.write("<count>1000</count>\n")
        f.write("<threads>30</threads>\n")
        f.write("<trecFormat>true</trecFormat>\n")
        f.write("</parameters>")
"""
<goodSTYs>
Clinical Drug
Disease or Syndrome
Injury or Poisoning
Pharmacologic Substance
Sign or Symptom
Therapeutic or Preventive Procedure
</goodSTYs>
"""

'\n<goodSTYs>\nClinical Drug\nDisease or Syndrome\nInjury or Poisoning\nPharmacologic Substance\nSign or Symptom\nTherapeutic or Preventive Procedure\n</goodSTYs>\n'

In [111]:
generateIndriQueries(cds14Indriconfigs, qWeightsLayerConcepts, indexDir, goodTui)

queryfile = /home/fj9124/projects/ir/seq_kb_ir/configs/cds14/indriRunQuery.cfg


In [113]:
with open(cds14Indriconfigs, 'r') as f:
    lines = f.read()
    print((lines)) 

<parameters>
<index>/scratch/index/indri_5_7/pmc_mod/</index>
<query>
<number>24</number>
<text>
	#weight(
		0.95 #combine( 33 year old male presents with severe abdominal pain one week after a bike accident in which he sustained abdominal trauma  He is hypotensive and tachycardic  and imaging reveals a ruptured spleen and intraperitoneal hemorrhage )
		0.05 #weight(
			1.0 #weight(
				0.8 #combine(severe pain)
				0.1 #combine(#od4(severe pain))
				0.1 #uw17(severe pain)
			)
			1.0 #weight(
				0.8 #combine(ruptured spleen)
				0.1 #combine(#od4(ruptured spleen))
				0.1 #uw17(ruptured spleen)
			)
			1.0 #weight(
				0.8 #combine(abdominal trauma)
				0.1 #combine(#od4(abdominal trauma))
				0.1 #uw17(abdominal trauma)
			)
			1.0 #weight(
				0.8 #combine(bike accident)
				0.1 #combine(#od4(bike accident))
				0.1 #uw17(bike accident)
			)
			1.0 #weight(
				0.8 #combine(abdominal pain)
				0.1 #combine(#od4(abdominal pain))
				0.1 #uw17(abdominal pain)
			)
		)
		0.01 #weight

In [ ]:
indriRunQueryRes = subprocess.check_output(["IndriRunQuery", cds14Indriconfigs])
print(indriRunQueryRes)